# Constants:

In [1]:
SEQ_LEN = 60   # how many past candles to use to predict
CANDLES_SHIFT = 1 # how many candles to shift between sequences
NAME = "decay_seq60_ov40th004"
VALIDATION_PCT = 0.2

DF initialisation:

In [1]:
import pandas as pd

df_300_classified = pd.read_csv("data/aligned/HistoricalDataClassified_2016_2023_300_ov40_th04.csv")
df_900 = pd.read_csv("data/aligned/HistoricalData_2016_2023_900.csv")
df_3600 = pd.read_csv("data/aligned/HistoricalData_2016_2023_3600.csv")
df_21600 = pd.read_csv("data/aligned/HistoricalData_2016_2023_21600.csv")


df_300_classified["timescale"] = 300
df_900["timescale"] = 900
df_3600["timescale"] = 3600
df_21600["timescale"] = 21600

print(df_300_classified)

         timestamp  BTC_close   BTC_low  BTC_high  BTC_volume  BTC_average  \
0       1467331500     670.90    668.00    670.90     7182.07       668.56   
1       1467331800     667.95    667.95    670.90     6205.90       670.89   
2       1467332100     671.67    670.95    672.99     6725.32       670.95   
3       1467332400     671.67    671.67    673.50      212.53       672.25   
4       1467332700     673.00    673.00    673.00      484.56       673.00   
...            ...        ...       ...       ...         ...          ...   
710104  1680362400   28423.92  28416.67  28440.89     4032.00     28424.22   
710105  1680362700   28402.64  28401.90  28426.65     1226.36     28409.15   
710106  1680363000   28418.48  28403.11  28418.48     4559.18     28406.68   
710107  1680363300   28412.62  28407.49  28422.84      388.83     28413.01   
710108  1680363600   28422.07  28384.76  28422.09      306.02     28407.02   

        BTC_HLPercent  target  timescale  
0            0.00432

# Functions:

compose sequences df

balance

In [5]:
def balance(dfs):
    
    buys = []
    sells = []
    holds = []
    for df in dfs:
        if df.at[len(df)-1, 'target'] == 0:
            sells.append(df)
        elif df.at[len(df)-1, 'target'] == 1:
            buys.append(df)
        else:
          holds.append(df)

    print("before balancing:")
    print("buys:", len(buys), ", sells:", len(sells), ", holds:", len(holds))

    smallest = min(len(buys), len(sells), len(holds))
    buys = buys[:smallest]
    sells = sells[:smallest]
    holds = holds[:smallest]

    dfsBalanced = buys+sells+holds
    return dfsBalanced

preprocessing

In [6]:
import numpy as np

def new_preprocess(df):

    for col in ["BTC_close", "BTC_low", "BTC_high", "BTC_average"]:
        #print(col)
        df[col] = np.log(df[col])
        df[col] = df[col].pct_change()
        df.dropna(inplace=True)
        #mean = np.mean(df[col])
        mean = 0.00000068
        #std = np.std(df[col])
        std = 0.00028
        #print("mean:", mean, ", std:", std)
        df[col] = (df[col] - mean) / std
    
    
    df["BTC_volume"] = df["BTC_volume"].replace(0, 1)
    df["BTC_volume"] = np.log(df["BTC_volume"])
    #df["BTC_volume"] = df["BTC_volume"].pct_change()   # taking the pct change somehow makes it worse
    #df.dropna(inplace=True)                            # taking the pct change somehow makes it worse
    #mean = np.mean(df["BTC_volume"])
    mean = 9.3
    #std = np.std(df["BTC_volume"])
    std = 2.82
    #print("mean:", mean, ", std:", std)
    df["BTC_volume"] = (df["BTC_volume"] - mean) / std


    #mean = np.mean(df["BTC_HLPercent"])
    mean = 0.0027
    #std = np.std(df["BTC_HLPercent"])
    std = 0.0032
    #print("mean:", mean, ", std:", std)
    df["BTC_HLPercent"] = (df["BTC_HLPercent"] - mean) / std


    return df

sequences

In [7]:
def buildSequences(dfs):
    
    sequences = []
    for df in dfs:
        if(len(df) == SEQ_LEN):
            label = df.at[SEQ_LEN-1, 'target']
            df = df.iloc[:, :-1]
            dfArray = df.values.tolist()
            sequences.append([np.array(dfArray), label])
    
    return sequences

X y split

In [8]:
def split(seqWithTarget):

    X = []
    y = []
    for seq, target in seqWithTarget:
        X.append(seq)
        y.append(target)

    return np.array(X),np.array(y)

# DF manipulation, build training sets:

In [9]:
# new preprocessing before splitting

df_preprocessed = new_preprocess(main_df)
print(df_preprocessed)

        BTC_close   BTC_low  BTC_high  BTC_volume  BTC_average  BTC_HLPercent  \
4       -0.002429 -0.002429 -0.002429   -3.297872    -0.002429       1.481399   
5       -0.002429 -0.002429 -0.002429   -3.297872    -0.002429       1.481399   
6       -0.002429 -0.002429 -0.002429   -3.297872    -0.002429       1.481399   
7        0.376210  0.252769 -1.637556   -0.842769    -2.596816       0.410914   
8        2.083405  2.206908  0.816119   -0.129185     1.235813      -0.378027   
...           ...       ...       ...         ...          ...            ...   
711758  -0.205381  0.253832  0.002469   -0.353894     0.335073      -0.577628   
711759  -0.263258 -0.183495 -0.176833   -0.775955    -0.187120      -0.571668   
711760   0.191756  0.012409 -0.102535   -0.310320    -0.032711      -0.674736   
711761  -0.074250  0.051276  0.050999   -1.183283     0.075172      -0.674982   
711762   0.113388 -0.281215 -0.011618   -1.268209    -0.075859      -0.433308   

        target  
4         

split into dfs with SEQ_LEN rows

In [10]:
import numpy as np
#splittedDfs = splitDf(main_df)
splittedDfs = splitDf_new(main_df)


splitDf
-done



seperate training and validation

In [11]:
dfsTraining = splittedDfs[:(int(len(splittedDfs) * (1-VALIDATION_PCT)))].copy()
dfsValidation = splittedDfs[(int(len(splittedDfs) * (1-VALIDATION_PCT))):].copy()

shuffle

In [12]:
import random

random.shuffle(dfsTraining)
random.shuffle(dfsValidation)

balance buys/sells/holds

In [13]:
dfsTrainingBalanced = balance(dfsTraining)
dfsValidationBalanced = balance(dfsValidation) # balance(dfsValidation) , validation data does not have to be balanced   

before balancing:
buys: 69791 , sells: 65907 , holds: 433310
before balancing:
buys: 13178 , sells: 13263 , holds: 115811


shuffle

In [14]:
random.shuffle(dfsTrainingBalanced)
random.shuffle(dfsValidationBalanced)

preprocessing

In [43]:
#dfsTrainingPreprocessed = preprocess(dfsTrainingBalanced)
#dfsValidationPreprocessed = preprocess(dfsValidationBalanced)

build sequences

In [15]:
#sequencesTraining = buildSequences(dfsTrainingPreprocessed)
#sequencesValidation = buildSequences(dfsValidationPreprocessed)
sequencesTraining = buildSequences(dfsTrainingBalanced)
sequencesValidation = buildSequences(dfsValidationBalanced)

shuffle

In [16]:
random.shuffle(sequencesTraining)
random.shuffle(sequencesValidation)

split sequence from label

In [17]:
train_x, train_y = split(sequencesTraining)
validation_x, validation_y = split(sequencesValidation)

In [21]:
print(train_x[0])

[[ 0.91136879  0.45031128  0.62217201 -0.11895851  0.23733937 -0.32841442]
 [ 0.22624879  0.52021516  0.23093835  0.1606478   0.74183958 -0.5360634 ]
 [-0.23578307 -0.58375053 -0.00594551  0.48329493 -0.06223915 -0.12131287]
 ...
 [-2.08518304 -2.00941311 -0.51880486  0.61353161 -1.34881858  0.66544726]
 [ 3.89597751  0.22294648  1.95445731  0.38947661 -0.15769953  1.91467241]
 [-0.9875773   2.28206724  1.03308996  0.51533883  3.5612343   1.0154201 ]]


In [22]:
import pickle

with open(f'data/pkl/{NAME}_train_x.pkl', 'wb') as file:
    pickle.dump(train_x, file)
with open(f'data/pkl/{NAME}_train_y.pkl', 'wb') as file:
    pickle.dump(train_y, file)
with open(f'data/pkl/{NAME}_validation_x.pkl', 'wb') as file:
    pickle.dump(validation_x, file)
with open(f'data/pkl/{NAME}_validation_y.pkl', 'wb') as file:
    pickle.dump(validation_y, file)

In [2]:
import pickle

with open(f"data/pkl/{NAME}_train_x.pkl", "rb") as file:
    train_x = pickle.load(file)
with open(f"data/pkl/{NAME}_train_y.pkl", "rb") as file:
    train_y = pickle.load(file)
with open(f"data/pkl/{NAME}_validation_x.pkl", "rb") as file:
    validation_x = pickle.load(file)
with open(f"data/pkl/{NAME}_validation_y.pkl", "rb") as file:
    validation_y = pickle.load(file)

# Model:

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [4]:
EPOCHS = 50

batchSize = 32
layers = 2
nodes = 256
denseNodes = 128
dropOut = 0.8
learningRate = 0.0001
decay = 1e-06


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from tqdm.keras import TqdmCallback


# model
model = Sequential()

for _ in range(layers-1):
  model.add(LSTM(nodes, activation="tanh", recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, input_shape=(train_x.shape[1:]), return_sequences=True))
  model.add(Dropout(dropOut))
  model.add(BatchNormalization())

model.add(LSTM(nodes, activation="tanh", recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, input_shape=(train_x.shape[1:])))
model.add(Dropout(dropOut))
model.add(BatchNormalization())

model.add(Dense(denseNodes, activation="relu"))
model.add(Dropout(dropOut))

model.add(Dense(3, activation="softmax"))



# opt  
opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?


model.compile(loss="sparse_categorical_crossentropy",
              optimizer = opt,
              metrics=["accuracy"])


filename = NAME + "-{epoch:02d}"
filepath = f"models/{filename}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch


# train
history = model.fit(
  train_x, train_y,
  batch_size = batchSize,
  epochs = EPOCHS,
  validation_data=(validation_x, validation_y),
  callbacks=[checkpoint, TqdmCallback(verbose=1)])
  #callbacks=[checkpoint])

c:\Users\Johannes\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/50
6179/6179 [==============================] - 539s 87ms/step - loss: 1.2139 - accuracy: 0.4959 - val_loss: 0.9171 - val_accuracy: 0.5901
Epoch 2/50
6179/6179 [==============================] - 534s 86ms/step - loss: 0.9367 - accuracy: 0.5811 - val_loss: 0.9002 - val_accuracy: 0.6029
Epoch 3/50
6179/6179 [==============================] - 530s 86ms/step - loss: 0.9111 - accuracy: 0.5978 - val_loss: 0.8945 - val_accuracy: 0.6015
Epoch 4/50
6179/6179 [==============================] - 530s 86ms/step - loss: 0.8991 - accuracy: 0.6044 - val_loss: 0.8907 - val_accuracy: 0.6042
Epoch 5/50
6179/6179 [==============================] - 530s 86ms/step - loss: 0.8928 - accuracy: 0.6081 - val_loss: 0.8921 - val_accuracy: 0.5994
Epoch 6/50
6179/6179 [==============================] - 536s 87ms/step - loss: 0.8871 - accuracy: 0.6121 - val_loss: 0.9025 - val_accuracy: 0.5946
Epoch 7/50
6179/6179 [==============================] - 536s 87ms/step - loss: 0.8797 - accuracy: 0.6162 - val_loss: 0

KeyboardInterrupt: 